Name: Daniel Masters ||
GitHub username: damasters ||
USC ID: 6203966352

<h1> DSCI 552 Homework #1 </h1>
<h2> 1. Vertebral Column Data Set</h2>
<p1> This Biomedical data set was built by Dr. Henrique da Mota during a medical residence
period in Lyon, France. Each patient in the data set is represented in the data set
by six biomechanical attributes derived from the shape and orientation of the pelvis
and lumbar spine (in this order): pelvic incidence, pelvic tilt, lumbar lordosis angle,
sacral slope, pelvic radius and grade of spondylolisthesis. The following convention is
used for the class labels: DH (Disk Hernia), Spondylolisthesis (SL), Normal (NO) and
Abnormal (AB). In this exercise, we only focus on a binary classification task NO=0
and AB=1. </p1>

(a)
Download the Vertebral Column Data Set from:
https://archive.ics.uci.edu/ml/datasets/Vertebral+Column

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import inf
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
dat_file = '../data/vertebral_column_data/column_2C.dat'

In [3]:
f = open(dat_file, "r")

FileNotFoundError: [Errno 2] No such file or directory: '../data/vertebral_column_data/column_2C.dat'

In [ ]:
d = pd.read_table(dat_file, header= None, delimiter = "\s+")
d.columns = ['pelvic incidence', 'pelvic tilt', 'lumbar lordosis angle', 'sacral slope', 'pelvic radius', 'grade of spondylolisthesis', 'class'] 
df = pd.DataFrame(data=d)
df['class'] = df['class'].replace('AB', 1)
df['class'] = df['class'].replace('NO', 0)
df.to_csv('DSCI_552_HW1_df.csv', index = False)

In [ ]:
df_csv = pd.read_csv('DSCI_552_HW1_df.csv')

(b) Pre-Processing and Exploratory data analysis:
 i) Make scatterplots of the independent variables in the dataset. Use color to
    show Classes 0 and 1.

In [ ]:
sns.set()
sns.pairplot(df_csv, hue= 'class', vars= ['pelvic incidence', 'pelvic tilt', 'lumbar lordosis angle', 'sacral slope', 'pelvic radius', 'grade of spondylolisthesis'])
plt.show()

b)ii) Make boxplots for each of the independent variables. Use color to show
Classes 0 and 1 (see ISLR p. 129).

In [ ]:
sns.boxplot(x= 'class', y= 'pelvic incidence', data= df_csv, hue= 'class')

In [ ]:
sns.boxplot(x= 'class', y= 'pelvic tilt', data= df_csv, hue= 'class')

In [ ]:
sns.boxplot(x= 'class', y= 'lumbar lordosis angle', data= df_csv, hue= 'class')

In [ ]:
sns.boxplot(x= 'class', y= 'sacral slope', data= df_csv, hue= 'class')

In [ ]:
sns.boxplot(x= 'class', y= 'pelvic radius', data= df_csv, hue= 'class')

In [ ]:
sns.boxplot(x= 'class', y= 'grade of spondylolisthesis', data= df_csv, hue= 'class')

b)iii)
Select the first 70 rows of Class 0 and the first 140 rows of Class 1 as the
training set and the rest of the data as the test set.


In [ ]:
train_class_0 = df[df['class'] == 0].iloc[:70, :]
train_class_1 = df[df['class'] == 1].iloc[:140, :]
dfs = [train_class_0, train_class_1]
train_set = pd.concat(dfs)
train_set

In [ ]:
#creating training sets with and without labels to feed algorithms
train_set_x = train_set.iloc[:, 0:6]
train_set_y = train_set.iloc[:, 6]

In [ ]:
other_vals_0 = df[df['class'] == 0].iloc[70:, :]
other_vals_1 = df[df['class'] == 1].iloc[140:, :]
dfs_2 = [other_vals_0, other_vals_1]
test_set = pd.concat(dfs_2)
test_set

In [ ]:
#creating test sets with and without labels to feed algorithms
test_set_x = test_set.iloc[:, :6]
test_set_y = test_set.iloc[:, 6]

(c) Classification using KNN on Vertebral Column Data Set

c)i) Write code for k-nearest neighbors with Euclidean metric (or use a software
package).

c)ii) Test all the data in the test database with k nearest neighbors. Take decisions by majority polling. Plot train and test errors in terms of k for k ∈ {208, 205, . . . , 7, 4, 1, } (in reverse order). You are welcome to use smaller increments of k. Which k* is the most suitable k among those values? Calculate the confusion matrix, true positive rate, true negative rate, precision, and F-score when k = k*.

In [ ]:
acc_list = {}
for k in range(208, 0, -1):
    knn = KNeighborsClassifier(n_neighbors = k) #210 training points
    knn.fit(train_set_x, train_set_y)    #fit to training data
    pred_k = knn.predict(test_set_x)     #prediction with test data
    pred_acc = accuracy_score(test_set_y, pred_k)*100
    acc_list[k] = pred_acc
best_k = max(acc_list, key= acc_list.get)
best_acc = max(acc_list.values())
print("The most suitable K is:",str(best_k),"\nIt's accuracy is: "+str(best_acc)+"%")
    

In [ ]:
#creating a confusion matrix with k=4
knn_4= KNeighborsClassifier(n_neighbors= 4)
knn_4.fit(train_set_x, train_set_y)
pred_k_4= knn_4.predict(test_set_x)
conf_matrix= confusion_matrix(test_set_y, pred_k_4)
print("Confusion Matrix calculation:\n"+str(conf_matrix))

In [ ]:
#getting the precision, recall, f1-score
print(classification_report(test_set_y, pred_k_4))

In [ ]:
#calculating true positive rate = TP/TP+FN
tp = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
tp_percent = tp*100
print("The true positive rate is: "+str(tp)+" or "+str(tp_percent)+"%")

In [ ]:
#calculating true negative rate = TN/FP+TN
tn = conf_matrix[1,1] / (conf_matrix[0,1] + conf_matrix[1,1])
tn_percent = tn*100
print("The true negative rate is: "+str(tn)+" or "+str(tn_percent)+"%")

In [ ]:
#calculating the precision = TP/TP+FP
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
prec_percent = precision*100
print("The precision is: "+str(precision)+" or "+str(prec_percent)+"%")

In [ ]:
#calculating the F1-score = TP / (TP + .5(FP + FN))
f1_score = conf_matrix[0,0] / (conf_matrix[0,0] + .5*(conf_matrix[0,1] + conf_matrix[1,0]))
print("The f1-score is: "+str(f1_score))

In [ ]:
#plotting train and test errors of k
x_axis = []
tr_error_list = []
te_error_list = []
for k in range(210, 0, -1):
    x_axis.append(k)
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(train_set_x, train_set_y)
    pred_train = knn.predict(train_set_x)
    pred_test = knn.predict(test_set_x)
    training_error = (1 - accuracy_score(train_set_y, pred_train))
    tr_error_list.append(training_error)
    test_error = (1 - accuracy_score(test_set_y, pred_test))
    te_error_list.append(test_error)
plt.plot(x_axis, tr_error_list, label = "Training Error")
plt.plot(x_axis, te_error_list, label = "Test Error")
plt.title("Training and Test Errors at Each K")
plt.xlabel("Number of Neighbors at Each K")
plt.ylabel("Error (in Hundredths)")
plt.legend()
plt.show()

c)iii) Since the computation time depends on the size of the training set, one may only use a subset of the training set. Plot the best test error rate, which is obtained by some value of k, against the size of training set, when the size of training set is N ∈ {10, 20, 30, . . . , 210}. Note: for each N, select your training set by choosing the ﬁrst N/3 rows of Class 0 and the ﬁrst N − N/3 rows of Class 1 in the training set you created in 1(b)iii. Also, for each N, select the optimal k from a set starting from k = 1, increasing by 5. For example, if N = 200, the optimal k is selected from {1, 6, 11, . . . , 196}. This plot is called a Learning Curve.

In [ ]:
n_list = []
best_error_list = []
for n in range(10, 211, 10):
    n_list.append(n)
    calc_0= math.floor(n/3)
    calc_1= n - (math.floor(n/3))
    train_class_0_sub= train_class_0.iloc[:calc_0]
    train_class_1_sub= train_class_1.iloc[:calc_1]
    combined= pd.concat([train_class_0_sub, train_class_1_sub])
    new_train_x= combined.iloc[:, 0:6]
    new_train_y= combined.iloc[:, 6]
    optimal_k = []
    for k in range(1, n, 5):
        knn_k= KNeighborsClassifier(n_neighbors= k)
        knn_k.fit(new_train_x, new_train_y)
        new_pred= knn_k.predict(test_set_x)
        new_acc= accuracy_score(test_set_y, new_pred)
        error_rate= 1-new_acc
        optimal_k.append(error_rate)
    best_error= min(optimal_k)
    best_error_list.append(best_error)
plt.plot(n_list, best_error_list, label="Best Test Error Rate")
plt.title("Plotting Best Test Error Rate")
plt.xlabel("Size of Training Set (N)")
plt.ylabel("Best Test Error Rate")
plt.legend()
plt.show()

(d) Replace the Euclidean metric with the following metrics and test them. Summarize the test errors (i.e., when k = k∗) in a table. Use all of your training data and select the best k when {1, 6, 11, . . . , 196}.

d)i) Minkowski Distance:
    A. which becomes Manhattan Distance with p= 1.

In [ ]:
error_dict = {}
for k in range(1, 197, 5):
    knn = KNeighborsClassifier(n_neighbors= k, p= 1) #210 training points
    knn.fit(train_set_x, train_set_y)    #fit to training data
    pred_k = knn.predict(test_set_x)     #prediction with test data
    pred_acc = accuracy_score(test_set_y, pred_k)
    pred_error= 1-pred_acc
    error_dict[k] = pred_error
k_best = min(error_dict, key= error_dict.get)
best_error_rate = min(error_dict.values())
print("The best K is:",str(k_best),"\nIt's best error rate is: "+str(best_error_rate)+".")


d)i)B) with log(10)(p) ∈ {0.1, 0.2, 0.3, . . . , 1}. In this case, use the k∗ you found for the Manhattan distance in 1(d)iA. What is the best log(10)(p)?

In [ ]:
knn_6_errdict={}
for p in np.arange(0.1, 1.1, 0.1):
    knn_6= KNeighborsClassifier(n_neighbors= 6, p= pow(10,p))
    knn_6.fit(train_set_x, train_set_y)
    knn_6_predict= knn_6.predict(test_set_x)
    knn_6_acc= accuracy_score(test_set_y, knn_6_predict)
    knn_6_error= 1-knn_6_acc
    knn_6_errdict[p] = knn_6_error
best_p = min(knn_6_errdict, key= knn_6_errdict.get)
knn_6_best = min(knn_6_errdict.values())
print("The best log10(p) is:",str(best_p),"\nIt's best error rate is: "+str(knn_6_best)+".")

d)i)C) which becomes Chebyshev Distance with p→ ∞

In [ ]:
knn_errdict = {}
for i in range(1, 197, 5):
    knn= KNeighborsClassifier(n_neighbors= i, metric= 'chebyshev')
    knn.fit(train_set_x, train_set_y)
    knn_predict= knn.predict(test_set_x)
    knn_acc= accuracy_score(test_set_y, knn_predict)
    knn_error= 1-knn_acc
    knn_errdict[i] = knn_error
best_cheby_err = min(knn_errdict.values())
mult_k = []
for k, v in knn_errdict.items():
    if v == best_cheby_err:
        mult_k.append(k)
best_cheby_k = max(mult_k)
print("The best k is:",str(best_cheby_k),"\nIt's best error rate is: "+str(best_cheby_err)+".")

d)ii) Mahalanobis Distance.

In [ ]:
from sklearn.neighbors import DistanceMetric

In [ ]:
knn_errdict= {}
cov_train= train_set_x.cov()
pseudo_inv= np.linalg.pinv(cov_train)
for i in range(1, 197, 5):
    knn= KNeighborsClassifier(n_neighbors= i, metric= 'mahalanobis', metric_params= {'VI': pseudo_inv})
    knn.fit(train_set_x, train_set_y)
    knn_predict= knn.predict(test_set_x)
    knn_acc= accuracy_score(test_set_y, knn_predict)
    knn_error= 1-knn_acc
    knn_errdict[i] = knn_error
best_mahala_err = min(knn_errdict.values())
many_k = []
for k, v in knn_errdict.items():
    if v == best_mahala_err:
        many_k.append(k)
best_mahala_k = max(many_k)
print("The best k is:",str(best_mahala_k),"\nIt's best error rate is: "+str(best_mahala_err)+".")

In [ ]:
table_format = {'Distance Measurements':['Manhattan Distance (p=1)', 'log10(p)', 'Chebyshev Distance', 'Mahalanobis'], 'Best K (or P)':[k_best, best_p, best_cheby_k, best_mahala_k], 'Best Test Error': [best_error_rate, knn_6_best, best_cheby_err, best_mahala_err]}
table = pd.DataFrame(table_format)
table

(e) The majority polling decision can be replaced by weighted decision, in which the weight of each point in voting is inversely proportional to its distance from the query/test data point. In this case, closer neighbors of a query point will have a greater inﬂuence than neighbors which are further away. Use weighted voting with Euclidean, Manhattan, and Chebyshev distances and report the best test errors when k ∈ {1, 6, 11, 16, . . . , 196}.

In [ ]:
euclid_dict= {}
for i in range(1, 197, 5):
    knn= KNeighborsClassifier(n_neighbors= i, weights= 'distance') #Euclidean Distance
    knn.fit(train_set_x, train_set_y)
    knn_predict= knn.predict(test_set_x)
    knn_acc= accuracy_score(test_set_y, knn_predict)
    knn_error= 1-knn_acc
    euclid_dict[i]= knn_error
best_euclid_err = min(euclid_dict.values())
mult_k = []
for k, v in euclid_dict.items():
    if v == best_euclid_err:
        mult_k.append(k)
best_euclid_k= max(mult_k)
print("The best k is:",str(best_euclid_k),"\nIt's best error rate is: "+str(best_euclid_err)+".")

In [ ]:
manha_dict= {}
for i in range(1, 197, 5):
    knn_2= KNeighborsClassifier(n_neighbors= i, weights= 'distance', p=1) #Manhattan Distance
    knn_2.fit(train_set_x, train_set_y)
    knn_2_predict= knn_2.predict(test_set_x)
    knn_2_acc= accuracy_score(test_set_y, knn_2_predict)
    knn_2_error= 1-knn_2_acc
    manha_dict[i] = knn_2_error
best_manha_err = min(manha_dict.values()) 
mult_k = []
for k, v in manha_dict.items():
    if v == best_manha_err:
        mult_k.append(k)
best_manha_k= max(mult_k)
print("The best k is:",str(best_manha_k),"\nIt's best error rate is: "+str(best_manha_err)+".")

In [ ]:
cheb_dict= {}
for i in range(1, 197, 5):
    knn_3= KNeighborsClassifier(n_neighbors= i, metric= 'chebyshev', weights= 'distance') #Chebyshev Distance
    knn_3.fit(train_set_x, train_set_y)
    knn_3_predict= knn_3.predict(test_set_x)
    knn_3_acc= accuracy_score(test_set_y, knn_3_predict)
    knn_3_error= 1-knn_3_acc
    cheb_dict[i] = knn_3_error
best_cheb_err = min(cheb_dict.values()) 
mult_k = []
for k, v in cheb_dict.items():
    if v == best_cheb_err:
        mult_k.append(k)
best_cheb_k= max(mult_k)
print("The best k is:",str(best_cheb_k),"\nIt's best error rate is: "+str(best_cheb_err)+".")

(f) What is the lowest training error rate you achieved in this homework?

0, because when k= 1 the closest training sample is itself.  